In [2]:
import torch
from torch.autograd import Variable
from torch import nn, optim
import torch.nn.functional as F
import torchvision
from torchvision import transforms, datasets
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
import operator

In [6]:
# ToTensor will convert the image from [0, 255] ot [0, 1]
# Normalize will do normalize the image i.e. subtract the mean and divide by std
# ImageDataGenerator => Keras
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.1307,), (0.3081,))])

In [7]:
trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
# flow_from_directory => Keras
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

Processing...
Done!


In [10]:
testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
# flow_from_directory => Keras
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

In [303]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
#         # conv1 = 28x28x1 => 28x28x32
#         # pooling = 28x28x32 => 14x14x32
        
#         self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
#         # conv2 = 14x14x32 => 14x14x64
#         # pooling = 14x14x64 => 7x7x64
        
#         self.fc1 = nn.Linear(64*7*7, 1024)
#         self.fc2 = nn.Linear(1024, 10)

        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.Conv2d(32, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, padding=1),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2))
        
        self.classifier = nn.Sequential(
            nn.Linear(64*7*7, 512),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(512, 10))
        
        
    def forward(self, x):
#         x = F.max_pool2d(F.relu(self.conv1(x)), 2)
#         x = F.max_pool2d(F.relu(self.conv2(x)), 2)
#         print (x.size())
#         # reshape Variable
#         x = x.view(-1, 64*7*7)
#         x = F.relu(self.fc1(x))
#         x = F.dropout(x, training=self.training)
#         x = self.fc2(x)
#         return x

        x = self.layer1(x)
        x = self.layer2(x)
        # print (x.size())
        x = x.view(-1, 64*7*7)
        x = self.classifier(x)
        return x

In [304]:
net = Net()

In [305]:
net

Net (
  (layer1): Sequential (
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
    (3): ReLU ()
    (4): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  )
  (layer2): Sequential (
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (3): ReLU ()
    (4): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  )
  (classifier): Sequential (
    (0): Linear (3136 -> 512)
    (1): ReLU ()
    (2): Dropout (p = 0.5)
    (3): Linear (512 -> 10)
  )
)

In [306]:
net.parameters()

<generator object Module.parameters at 0x7f21f8738ca8>

In [307]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-3)

# Training

In [308]:
for epoch in range(1):
    for i, data in enumerate(trainloader):
        inputs, labels = data
        inputs = Variable(inputs)
        labels = Variable(labels)
        
        # forward + backward + optimize
        
        # zeroes the gradient buffers of all parameters
        optimizer.zero_grad()
        outputs = net(inputs)
        # calculate the loss
        loss = loss_function(outputs, labels)
        # backpropagation
        loss.backward()
        # Does the update after calculating the gradients
        optimizer.step()
        
        if (i+1) % 500 == 0: # print every 100 mini-batches
            print('[%d, %5d] loss: %.4f' % (epoch, i+1, loss.data[0]))

[0,   500] loss: 0.0198
[0,  1000] loss: 0.2518
[0,  1500] loss: 0.0322


# Testing

In [309]:
net.eval()
correct = 0
total = 0

In [310]:
for images, label in testloader:
    images = Variable(images)
    outputs = net(images)
    _, predicted = torch.max(outputs.data, dim=1)
    total += label.size(0)
    correct += (label == predicted).sum()

print ('Accuracy :', (100 * correct) / total)

Accuracy : 98.79


In [311]:
outputs.data



Columns 0 to 7 
 -4.5812  10.8070  -4.2378  -5.2805  -0.7986  -3.7527  -6.4029   0.1857
 -4.7889  -0.5834  10.1286   1.3074  -8.1259 -10.2483 -13.5310   5.9491
-10.2210   0.1188  -5.1186  13.6526 -11.1181   1.5492 -10.7495  -0.7498
-10.2744  -2.4135  -6.4498  -7.8775  15.8837  -3.3097  -7.1414  -3.2188
 -7.4263  -9.1671 -15.3454   1.4392 -13.0718  18.4936  -3.7754 -11.0524
 -0.6917  -9.9763  -4.6020  -6.9191  -4.1801   2.7401  16.0168 -16.3044
-10.2381  -1.7301  -2.1853  -0.2322  -9.0601  -8.7824 -19.6885  16.2550
 -4.2174  -7.4268  -2.6289   0.3553 -11.4076  -0.1164  -4.5327  -7.3156
 -4.5174 -12.7149 -10.9024  -3.6221   2.3291  -4.2100 -11.6047  -1.7489
  9.8210  -9.7329  -5.4297  -4.1643  -6.8910  -1.0691   1.2138  -6.8332
 -2.9848  10.7950  -2.7532  -5.6745  -2.1495  -3.3792  -4.5603  -1.5601
 -6.0610  -0.3677  16.7050   0.6652 -10.1941 -13.4020 -13.8482   5.7012
 -8.1113  -1.4425  -4.2123  13.2636 -10.7504   2.0812  -8.6057  -1.0086
-11.7277  -2.4632  -7.1987  -9.7317  15.8251  

In [312]:
torch.max(outputs.data, 1)

(
  10.8070
  10.1286
  13.6526
  15.8837
  18.4936
  16.0168
  16.2550
  14.9983
  13.9584
   9.8210
  10.7950
  16.7050
  13.2636
  15.8251
  15.1088
  14.5902
 [torch.FloatTensor of size 16x1], 
     1
     2
     3
     4
     5
     6
     7
     8
     9
     0
     1
     2
     3
     4
     5
     6
 [torch.LongTensor of size 16x1])